<a href="https://colab.research.google.com/github/Achyuta-Harshavardhan/Keras_Tunner_ANN/blob/main/Keras_Tunner_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Keras Tunner for ANN**

In [1]:
# Install Keras tunner
!pip install -U keras-tuner

In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from kerastuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Deep Learning/Dataset/Real_Combine.csv')
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [4]:
# Independet Fetures
x = df.iloc[:,:-1]

# Dependent feature
y = df.iloc[:,-1]

**Hyperparameters:**
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [5]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [6]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    directory='project',
    project_name='Air Quality Index')

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [8]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_mean_absolute_error: 61.57698059082031

Best val_mean_absolute_error So Far: 56.99097442626953
Total elapsed time: 00h 00m 12s
INFO:tensorflow:Oracle triggered exit


In [9]:
model=tuner.get_best_models(num_models=1)[0]

In [10]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

24/24 [==============================] - 0s 13ms/step - loss: nan - mean_absolute_error: nan - val_loss: 56.1178 - val_mean_absolute_error: 56.1178
Epoch 2/10
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 45.6494 - val_mean_absolute_error: 45.6494
Epoch 3/10
24/24 [==============================] - 0s 5ms/step - loss: nan - mean_absolute_error: nan - val_loss: 93.1872 - val_mean_absolute_error: 93.1872
Epoch 4/10
24/24 [===============